In [0]:
dbutils.widgets.dropdown(
    name="environment",
    defaultValue="fq_dev",
    choices=["fq_dev", "fq_test", "fq_prod"],
    label="Select environment"
)

dbutils.widgets.combobox(
    name="source",
    defaultValue="excel_sheet",
    choices=["posist", "netsuite", "other","excel_sheet"],
    label="Source"
)

dbutils.widgets.combobox(
    name="domain",
    defaultValue="HR_workforce",
    choices=["discount", "sales", "cost","wastage","HRworkforce"],
    label="Domain"
)

environment = dbutils.widgets.get("environment")
source = dbutils.widgets.get("source")
domain = dbutils.widgets.get("domain")

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, to_date, concat_ws, last_day, expr, explode, row_number
from pyspark.sql.window import Window
from delta.tables import DeltaTable
import sys

def get_external_location(name: str) -> str:
    return (spark.sql(f"DESCRIBE EXTERNAL LOCATION `{name}`")
             .select("url")
             .collect()[0][0]
    )

bronze_path = get_external_location(f"{environment}_extloc_bronze")
silver_path = get_external_location(f"{environment}_extloc_silver")
gold_path = get_external_location(f"{environment}_extloc_gold")
checkpoint_path = get_external_location(f"{environment}_extloc_checkpoint")
staging_path = get_external_location(f"{environment}_extloc_staging")

In [0]:
from delta.tables import DeltaTable
from pyspark.sql.functions import col, current_timestamp, max as _max

SOURCE_TABLE = "fq_dev_catalog.bronze.hrworkforce"
VERSION_TABLE = "fq_dev_catalog.bronze.version_control"
CONSUMER_NAME = "silver_HR_workforce"

# 1️⃣ Get latest version from source
delta_table = DeltaTable.forName(spark, SOURCE_TABLE)

latest_version = (
    delta_table.history()
    .select(_max("version").alias("latest_version"))
    .collect()[0]["latest_version"]
)

# 2️⃣ Get last processed version
if spark.catalog.tableExists(VERSION_TABLE):
    last_processed_version = (
        spark.table(VERSION_TABLE)
        .filter(col("source_table") == SOURCE_TABLE)
        .filter(col("consumer_name") == CONSUMER_NAME)
        .agg(_max("last_processed_version").alias("last_version"))
        .collect()[0]["last_version"]
    )
else:
    last_processed_version = None

last_processed_version = last_processed_version if last_processed_version is not None else -1

# 3️⃣ Decide whether to process
if last_processed_version >= latest_version:
    print(
        f"[INFO] No new data to process | "
        f"Latest version: {latest_version}, "
        f"Last processed: {last_processed_version}"
    )
    df_bronze_workforce= None   # 👈 explicit
else:
    start_version = last_processed_version + 1
    print(
        f"[INFO] Processing versions from {start_version} to {latest_version}"
    )

    df_bronze_workforce = (
        spark.read
        .format("delta")
        .option("startingVersion", start_version)
        .table(SOURCE_TABLE)
    )

In [0]:
# df_bronze_workforce.select(col("rescued_data")).filter(col("rescued_data").isNotNull()).display()

In [0]:
if df_bronze_workforce is None or df_bronze_workforce.isEmpty():
    print("[INFO] No new wastage data to process — exiting job")
    dbutils.notebook.exit("NO_NEW_DATA") 

In [0]:
# df_bronze_workforce.display()

In [0]:
# from pyspark.sql.functions import count
# df_bronze_workforce.agg(count("*")).display()

In [0]:
from pyspark.sql.functions import element_at, split, col
df_bronze_workforce=df_bronze_workforce.withColumn("deployment_name",element_at(split(col("file_path"), "/"), -2))


In [0]:
# df_bronze_workforce.display()

In [0]:
store_df = (
        spark.read
            .table(f"{environment}_catalog.silver.dim_store")
            .select(
            "deployment_name"
        )
    )

In [0]:
from pyspark.sql.functions import lit, current_timestamp, col

workforce_rejected_stores = (
    df_bronze_workforce.alias("b")
    .join(
        store_df.alias("s"),
        col("b.deployment_name") == col("s.deployment_name"),
        "left"
    )
    .filter(col("s.deployment_name").isNull())
    .select("b.*")   # keep only budget columns
    .withColumn(
        "rejection_reason",
        lit("STORE_NOT_FOUND_IN_MASTER_TABLE_dim_store")
    )
    .withColumn("rejection_ts", current_timestamp())
)

In [0]:
# spark.sql(f"""
# CREATE TABLE IF NOT EXISTS fq_dev_catalog.silver.workforce_rejected_stores
# USING DELTA
# TBLPROPERTIES (delta.enableChangeDataFeed = true, delta.autoOptimize.optimizeWrite = true, delta.autoOptimize.autoCompact = true, delta.columnMapping.mode = 'name')
# """)

In [0]:
target_table="fq_dev_catalog.silver.workforce_rejected_stores"
def write_workforce_rejected_to_silver(df):
    query = (
            workforce_rejected_stores.write
                .format("delta")
                .mode("overwrite")
                .option("mergeSchema", "true")
                .saveAsTable(target_table)
    )
    return query
write_workforce_rejected_to_silver(workforce_rejected_stores)

In [0]:
# %sql
# select * from fq_dev_catalog.silver.workforce_rejected_stores

In [0]:
store_df = store_df.withColumnRenamed("deployment_name", "store_deployment_name")

def create_final_workforce(df, store_df):
    """
    Joins budget with dimensions and selects final columns.
    """
    df = df.join(
        store_df,
        df.deployment_name == store_df.store_deployment_name,
        "inner"
    )

    df= df.select("date","leave_categories","rhc_count","rhc_value","rc_count","rc_value","qcc_count","qcc_value","rac_count","rac_value","pac_count","pac_value","sac_count","sac_value","ltm_count","ltm_value","tm_count","tm_value","crl_count","crl_value","trainee_count","trainee_value","deployment_name","ingestion_ts","file_path")
    return df

df_bronze_base=create_final_workforce(df_bronze_workforce, store_df)

In [0]:
# df_bronze_base.display()

In [0]:
# df_bronze_base.agg(count("*")).display()

In [0]:
from pyspark.sql.functions import col, to_date, regexp_replace, date_format,lit

df_bronze_struct = (
    df_bronze_base
    .withColumn(
        "business_date",
        date_format(
            to_date(
                regexp_replace(col("Date"), r"\s*\(.*\)", ""),  # removes (Fri)
                "dd-MMM-yyyy"
            ),
            "yyyy-MM-dd"
        )
    )
    .withColumnRenamed("Leave_Categories", "task_category")
    .withColumn("source_system", lit("FoodQuest_sharepoint"))
    .drop("date")
)


In [0]:
# df_bronze_struct.display()

In [0]:
from pyspark.sql.functions import row_number, last, col
from pyspark.sql.window import Window

# Window for row numbering (FIXED)
w_row = (
    Window
    .partitionBy("deployment_name")   # 👈 IMPORTANT
    .orderBy("file_path")             # or _metadata.file_path
)

df = df_bronze_struct.withColumn(
    "row_num",
    row_number().over(w_row)
)


In [0]:
w_fill = (
    Window
    .partitionBy("deployment_name")
    .orderBy("row_num")
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)
)

df_filled = (
    df
    .withColumn(
        "business_date",
        last(col("business_date"), ignorenulls=True).over(w_fill)
    )
    .drop("row_num")
)


In [0]:
# df_filled.display()

In [0]:
from pyspark.sql.functions import expr

df_schema = df_filled.selectExpr(
    "business_date",
    "task_category",
    "deployment_name",
    "file_path",
    "source_system",
    """
        stack(
            10,
            'RHC', rhc_count, rhc_value,
            'RC',  rc_count,  rc_value,
            'QCC', qcc_count, qcc_value,
            'RAC', rac_count, rac_value,
            'PAC', pac_count, pac_value,
            'SAC', sac_count, sac_value,
            'LTM', ltm_count, ltm_value,
            'TM',  tm_count,  tm_value,
            'CRL', crl_count, crl_value,
            'Trainee', trainee_count, trainee_value
        ) as (designation, employee_count, employee_cost)
    """
)

df_schema.display()

In [0]:
# df_schema.agg(count("*")).display()

In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import DateType, StringType, IntegerType, DoubleType
from pyspark.sql.functions import col, trim, when

final_df = df_schema.select(
    col("business_date").cast(DateType()).alias("business_date"),
    "deployment_name",
    "task_category",
    "designation",

    col("employee_count").cast(IntegerType()).alias("employee_count"),

    when(
        trim(col("employee_cost")).isin("-", ""),
        None
    )
    .otherwise(trim(col("employee_cost")))
    .cast(DoubleType())
    .alias("employee_cost"),

    "file_path",
    "source_system"
)


In [0]:
# final_df.agg(count("*")).display()

In [0]:
# final_df.display()

In [0]:
# final_df.printSchema()

In [0]:
final_df_unique= final_df.dropDuplicates(["business_date","deployment_name","task_category","designation"])

In [0]:
# final_df_unique.agg(count("*")).display()

In [0]:
final_df_filled = final_df_unique.dropna(subset=["business_date", "deployment_name", "task_category", "designation"])

In [0]:
# final_df_filled.agg(count("*")).display()

In [0]:
# final_df_filled.agg(count("*")).display()

In [0]:
# final_df_filled.printSchema()

In [0]:
# from pyspark.sql.functions import sum as _sum, col, coalesce, lit

# aggregated_df_workforce = (
#     final_df_filled
#     .groupBy(
#         "business_date",
#         "deployment_name",
#         "task_category",
#         "designation"
#     )
#     .agg(
#         _sum(coalesce(col("employee_count"), lit(0))).alias("total_employee_count"),
#         _sum(coalesce(col("employee_cost"), lit(0.0))).alias("total_employee_cost")
#     )
# )


In [0]:
# aggregated_df_workforce.display()

In [0]:
# aggregated_df_workforce.agg(count("*")).display()

In [0]:
# %sql
# CREATE TABLE IF NOT EXISTS fq_dev_catalog.silver.hr_workforce (
#   business_date DATE ,
#   deployment_name STRING NOT NULL,
#   task_category STRING NOT NULL,
#   designation STRING NOT NULL,
#   employee_count INT ,
#   employee_cost DECIMAL(10,2),
#   file_path STRING NOT NULL,
#   source_system STRING NOT NULL
# )
# USING DELTA
# TBLPROPERTIES (
#    delta.enableChangeDataFeed = true,
#    delta.autoOptimize.optimizeWrite = true,
#    delta.autoOptimize.autoCompact = true,
#    delta.columnMapping.mode = 'name'
# );


In [0]:
from delta.tables import DeltaTable
import sys

def upsert_to_silver_batch(
    df,
    table_name,
    business_keys
):
    try:
        # Skip empty DataFrame
        if df.isEmpty():
            print("No data to upsert")
            return

        # Load target Delta table
        silver_table = DeltaTable.forName(spark, table_name)

        # Build merge condition dynamically
        merge_condition = " AND ".join(
            [f"t.{k} = s.{k}" for k in business_keys]
        )

        (
            silver_table.alias("t")
            .merge(
                df.alias("s"),
                merge_condition
            )
            .whenMatchedUpdate(set={
                "employee_count": "s.employee_count",
                "employee_cost": "s.employee_cost",
                "file_path": "s.file_path",
                "source_system": "s.source_system"
            })
            .whenNotMatchedInsert(values={
                "business_date": "s.business_date",
                "deployment_name":"s.deployment_name",
                "task_category": "s.task_category", 
                "designation" :"s.designation",
                "employee_count": "s.employee_count",
                "employee_cost": "s.employee_cost",
                "file_path": "s.file_path",
                "source_system": "s.source_system"
            })
            .execute()
        )

        print("Batch upsert to silver completed successfully")

    except Exception as e:
        print(f"Upsert to Silver failed: {e}")
        sys.exit(1)

In [0]:
silver_table_name = f"{environment}_catalog.silver.hr_workforce"

business_keys = [
        "business_date",
        "deployment_name",
        "task_category",
        "designation"
]

upsert_to_silver_batch(
    final_df_filled,        
    silver_table_name,       
    business_keys        
)

In [0]:
# %sql
# select * from fq_dev_catalog.silver.hr_workforce

In [0]:
# %sql
# select count("*") from fq_dev_catalog.silver.hr_workforce

In [0]:
# spark.sql(f"""
# INSERT INTO {VERSION_TABLE} (source_table, consumer_name, last_processed_version, updated_at)
# SELECT
#   '{SOURCE_TABLE}',
#   '{CONSUMER_NAME}',
#   {latest_version},
#   current_timestamp()
# WHERE NOT EXISTS (
#   SELECT 1
#   FROM {VERSION_TABLE}
#   WHERE
#     source_table = '{SOURCE_TABLE}'
#     AND consumer_name = '{CONSUMER_NAME}'
# )
# """)

In [0]:
spark.sql(f"""
UPDATE {VERSION_TABLE}
SET
  last_processed_version = {latest_version},
  updated_at = current_timestamp()
WHERE
  source_table = '{SOURCE_TABLE}'
  AND consumer_name = '{CONSUMER_NAME}'
""")